In [5]:
import pandas as pd

df1 = pd.read_csv("data.csv")
df2 = pd.read_csv("data.csv")

df = pd.concat([df1, df2])
df 

,Unnamed: 0,diamond_right-thalamus-AntCingCtx_wFA_mean,diamond_right-thalamus-AntCingCtx_wFA_std,diamond_right-thalamus-AntCingCtx_wFA_skew,diamond_right-thalamus-AntCingCtx_wFA_kurt,diamond_right-sup-longi-fasci_wFA_mean,diamond_right-sup-longi-fasci_wFA_std,diamond_right-sup-longi-fasci_wFA_skew,diamond_right-sup-longi-fasci_wFA_kurt,diamond_left-thalamus-Insula_wFA_mean,...,diamond_left-thalamus-AntCingCtx_wFA_skew,diamond_left-thalamus-AntCingCtx_wFA_kurt,diamond_left-amygdala_wFA_mean,diamond_left-amygdala_wFA_std,diamond_left-amygdala_wFA_skew,diamond_left-amygdala_wFA_kurt,diamond_right-amygdala_wFA_mean,diamond_right-amygdala_wFA_std,diamond_right-amygdala_wFA_skew,diamond_right-amygdala_wFA_kurt
0,0,0.483251,0.202976,-0.002161,0.004217,0.555738,0.13639,-0.001817,0.001117,0.497207,...,-0.004362,0.003049,0.304026,0.10012,0.000425,0.000349,0.295219,0.11574,0.000442,0.000621
0,0,0.483251,0.202976,-0.002161,0.004217,0.555738,0.13639,-0.001817,0.001117,0.497207,...,-0.004362,0.003049,0.304026,0.10012,0.000425,0.000349,0.295219,0.11574,0.000442,0.000621
